In [1]:
%matplotlib tk

In [2]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas
import numpy as np
import shapely
from typing import Optional

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
all_counties = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_500k.zip')

In [4]:
all_doctors = gpd.GeoDataFrame(
    pandas.read_csv(
        '/Users/eab06/Desktop/WJB/PythonProjects/HT_Data/data/processed/all_with_duplicates.csv',
        comment='#',
        index_col=0
    )
)

In [5]:
zipcodes = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_zcta510_500k.zip')

In [6]:
zipcodes.set_index('ZCTA5CE10', inplace=True)

In [7]:
def crop(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    return gdf.cx[-130:-68, :55] # crop to roughly 48 states

In [8]:
# approximate 48 states
counties = crop(all_counties)

In [9]:
counties = counties.to_crs(epsg=2163)

In [10]:
states = counties.dissolve('STATEFP')

In [11]:
country = states.dissolve(lambda _: True)

In [12]:
def closest_search(postal_code: int) -> Optional[shapely.geometry.Point]: # get closest centroid for zipcode
    try:
        geom = zipcodes.at[postal_code, 'geometry']
    except KeyError:
        int_idx = (zipcodes.index - postal_code).map(abs).argmin()
        if abs(int_idx - postal_code) > 10:
            return None
        idx_val = zipcodes.index[int_idx]
        geom = zipcodes.at[idx_val, 'geometry']

    return geom.centroid

zipcodes.index = zipcodes.index.astype(int)

# merge geometry into all_doctors by closest_search
all_doctors['geometry'] = all_doctors['postal_code']\
    .apply(lambda s: np.nan if pandas.isna(s) else int(s))\
    .apply(closest_search) 
all_doctors = all_doctors.set_crs(4326)

In [13]:
all_doctors = crop(all_doctors).to_crs(epsg=2163)

In [14]:
fig, ax = plt.subplots()
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.set_title("Data sources in the lower 48 states")

states.plot(fc='none', lw=0.5, ec='xkcd:dark gray', ax=ax)
counties.plot(color='none', lw=0.1, ec='gray', ax=ax)
all_doctors.plot(column='src', categorical=True, legend=True, cmap='brg', ax=ax, markersize=1)

fig.savefig('images/source_map.png', dpi=1000)

In [15]:
def get_county(zipcode_center: shapely.geometry.Point) -> Optional[str]:
    int_indices = counties.sindex.query(zipcode_center)
    if len(int_indices) == 0:
        return None
    idx = counties.index[int_indices[0]]
    return counties.at[idx, 'GEOID']
all_doctors['county_code'] = all_doctors.geometry.apply(get_county)

In [16]:
grouped_into_counties = gpd.GeoDataFrame(
    all_doctors\
    .groupby(['county_code', 'src'])\
    ['geometry']\
    .count()\
    .reset_index()\
    .rename(columns={'geometry': 'count'})\
    .merge(counties, left_on='county_code', right_on='GEOID')\
    [['geometry', 'count', 'src']]
)
grouped_into_counties.geometry = grouped_into_counties.geometry.centroid
    
grouped_into_counties

,geometry,count,src
0,POINT (1114623.102 -1063302.697),1,endocrinologists
1,POINT (1400534.512 -1415997.541),1,asoprs
2,POINT (1400534.512 -1415997.541),1,endocrinologists
3,POINT (1227176.357 -1039066.253),1,asoprs
4,POINT (1227176.357 -1039066.253),1,endocrinologists
...,...,...,...
796,POINT (890348.083 -199321.864),1,endocrinologists
797,POINT (964380.764 -70607.103),1,tepezza
798,POINT (946924.026 -152378.652),7,asoprs
799,POINT (946924.026 -152378.652),2,endocrinologists


In [17]:
fig2, ax2 = plt.subplots()
ax2.xaxis.set_visible(False)
ax2.yaxis.set_visible(False)
ax2.set_title("Data sources in the lower 48 states (aggregated by county)")

states.plot(fc='none', lw=0.5, ec='xkcd:dark gray', ax=ax2)
counties.plot(color='none', lw=0.1, ec='gray', ax=ax2)
grouped_into_counties.plot(column='src', categorical=True, legend=True, cmap='brg', ax=ax2, ec='none', alpha=0.2, markersize=grouped_into_counties['count'] * 20) # faded interiors
grouped_into_counties.plot(column='src', categorical=True, legend=True, cmap='brg', ax=ax2, fc='none', alpha=0.5, markersize=grouped_into_counties['count'] * 20) # edges

fig2.savefig('images/source_map_county_agg.png', dpi=1000)